In [ ]:
# ====================================================================================
# 1. IMPORTAR LIBRERÍAS
# ====================================================================================

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
from scipy import stats
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

# Directorios
DATA_DIR = Path('../data')
ERA5_PATH = DATA_DIR / 'processed' / 'era5_precipitation_chile_full.nc'
CHIRPS_PATH = DATA_DIR / 'external' / 'chirps' / 'chirps_chile_2019-01-01_2020-02-29.nc'
FORECAST_PATH = DATA_DIR / 'processed' / 'forecast_results_2020.pkl'
FIG_DIR = Path('../reports/figures')
FIG_DIR.mkdir(parents=True, exist_ok=True)

print("📦 Librerías importadas correctamente")
print(f"   ERA5: {ERA5_PATH.exists()}")
print(f"   CHIRPS: {CHIRPS_PATH.exists()}")
print(f"   Forecast: {FORECAST_PATH.exists()}")

In [ ]:
# ====================================================================================
# 2. CARGAR DATOS
# ====================================================================================

# Cargar ERA5 (datos de entrenamiento/validación del modelo)
print("📥 Cargando datos ERA5...")
ds_era5 = xr.open_dataset(ERA5_PATH)
print(f"   Variables: {list(ds_era5.data_vars)}")
print(f"   Coordenadas: {list(ds_era5.coords)}")
print(f"   Shape: {ds_era5['tp'].shape if 'tp' in ds_era5 else 'N/A'}")

# Cargar CHIRPS (datos satelitales para validación)
if CHIRPS_PATH.exists():
    print("\n📥 Cargando datos CHIRPS...")
    ds_chirps = xr.open_dataset(CHIRPS_PATH)
    print(f"   Variables: {list(ds_chirps.data_vars)}")
    print(f"   Coordenadas: {list(ds_chirps.coords)}")
    print(f"   Shape: {ds_chirps['precip'].shape}")
    print(f"   Periodo: {ds_chirps.time.values[0]} a {ds_chirps.time.values[-1]}")
else:
    print("\n⚠️ Datos CHIRPS no disponibles")
    print("   Ejecutar: cd src/utils && python download_chirps.py")
    ds_chirps = None

# Cargar predicciones del modelo
print("\n📥 Cargando predicciones AE+DMD...")
with open(FORECAST_PATH, 'rb') as f:
    forecast_results = pickle.load(f)
    
y_test_real = forecast_results['y_test_real']
h1_preds = forecast_results['forecast_results'][1]['predictions']
print(f"   Ground truth (ERA5 test): {y_test_real.shape}")
print(f"   Predicciones h=1: {h1_preds.shape}")

In [ ]:
# ====================================================================================
# 3. ALINEAR TEMPORALMENTE ERA5 Y CHIRPS
# ====================================================================================

if ds_chirps is not None:
    print("🔗 Alineando ERA5 y CHIRPS temporalmente...")
    
    # Identificar periodo común (ej: test 2020)
    # El test set son 55 días de 2020 (01-ene a 24-feb)
    test_start = '2020-01-01'
    test_end = '2020-02-24'
    
    # Extraer periodo de test de CHIRPS
    chirps_test = ds_chirps.sel(time=slice(test_start, test_end))
    
    print(f"   Periodo de test: {test_start} a {test_end}")
    print(f"   CHIRPS test shape: {chirps_test['precip'].shape}")
    print(f"   ERA5 test shape: {y_test_real.shape}")
    
    # Verificar resoluciones
    print(f"\n📐 Resoluciones espaciales:")
    print(f"   CHIRPS: {chirps_test.latitude.shape[0]} lat x {chirps_test.longitude.shape[0]} lon")
    print(f"   ERA5: {y_test_real.shape[1]} lat x {y_test_real.shape[2]} lon")
    
else:
    print("⚠️ Saltando alineación (CHIRPS no disponible)")

In [ ]:
# ====================================================================================
# 4. INTERPOLAR CHIRPS A RESOLUCIÓN ERA5
# ====================================================================================

if ds_chirps is not None:
    print("🔄 Interpolando CHIRPS a resolución ERA5...")
    
    # CHIRPS: 0.05° → ERA5: 0.25° (necesita agregación o interpolación)
    # Método: Regrid usando xarray
    
    # Obtener grilla de ERA5 (asumiendo que está en ds_era5 o forecast_results)
    # Por ahora, usaremos interpolación simple
    
    # TODO: Implementar regridding correcto
    # chirps_regridded = chirps_test.interp(
    #     latitude=era5_lat,
    #     longitude=era5_lon,
    #     method='linear'
    # )
    
    print("   ⚠️ Interpolación pendiente de implementación")
    print("   Se requiere conocer coordenadas exactas ERA5 del proyecto")
    
else:
    print("⚠️ Saltando interpolación (CHIRPS no disponible)")

In [ ]:
# ====================================================================================
# 5. COMPARACIÓN ERA5 vs CHIRPS (Ground Truth)
# ====================================================================================

if ds_chirps is not None:
    print("📊 Comparando ERA5 vs CHIRPS (ground truth)...")
    
    # Métricas de similitud entre ERA5 y CHIRPS
    # Esto valida que ERA5 es representativo de precipitación real
    
    # TODO: Calcular correlación espacial, RMSE, bias
    # para cada día del test set
    
    print("   ⚠️ Comparación pendiente de implementación")
    print("   Métricas a calcular:")
    print("   - Correlación espacial por día")
    print("   - RMSE y MAE entre ERA5 y CHIRPS")
    print("   - Bias maps (ERA5 - CHIRPS)")
    print("   - Scatter plots agregados por macrozona")
    
else:
    print("⚠️ Saltando comparación (CHIRPS no disponible)")
    print("\n💡 Una vez descargados los datos CHIRPS, este notebook permitirá:")
    print("   1. Validar que ERA5 representa bien la precipitación real (vs CHIRPS)")
    print("   2. Evaluar predicciones AE+DMD contra dos fuentes independientes")
    print("   3. Identificar regiones con mayor/menor confiabilidad")

In [ ]:
# ====================================================================================
# 6. COMPARACIÓN PREDICCIONES AE+DMD vs CHIRPS
# ====================================================================================

if ds_chirps is not None:
    print("🎯 Evaluando predicciones AE+DMD contra CHIRPS...")
    
    # Comparar h1_preds (predicciones modelo) vs CHIRPS interpolado
    # Esto responde: ¿El modelo predice bien vs datos satelitales independientes?
    
    # TODO: Calcular métricas de predicción
    # MAE, RMSE, correlación por macrozona
    
    print("   ⚠️ Evaluación pendiente de implementación")
    
else:
    print("⚠️ Saltando evaluación (CHIRPS no disponible)")

In [ ]:
# ====================================================================================
# 7. VISUALIZACIONES
# ====================================================================================

# Placeholder para visualizaciones cuando CHIRPS esté disponible
print("📊 Visualizaciones planificadas:")
print("   1. Mapas comparativos: ERA5 vs CHIRPS vs Predicciones")
print("   2. Series temporales por macrozona (3 fuentes)")
print("   3. Scatter plots: ERA5 vs CHIRPS, Pred vs CHIRPS")
print("   4. Mapas de bias: (ERA5 - CHIRPS), (Pred - CHIRPS)")
print("   5. Distribuciones acumuladas por región")
print("\n💡 Ejecutar primero: src/utils/download_chirps.py")

In [ ]:
# ====================================================================================
# 8. RESUMEN Y CONCLUSIONES
# ====================================================================================

print("\n" + "="*80)
print("RESUMEN - Validación CHIRPS")
print("="*80)

if ds_chirps is None:
    print("\n⚠️ DATOS CHIRPS NO DISPONIBLES")
    print("\nPara completar esta validación:")
    print("   1. Ejecutar: cd src/utils && python download_chirps.py")
    print("   2. Esperar descarga (~2-4 GB, puede tardar varios minutos)")
    print("   3. Re-ejecutar este notebook")
    print("\nAlternativas si la descarga falla:")
    print("   - FTP manual: ftp://ftp.chc.ucsb.edu/pub/org/chg/products/CHIRPS-2.0/")
    print("   - Google Earth Engine: https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY")
    print("   - Climate Engine: https://climateengine.org/")
else:
    print("\n✅ DATOS CHIRPS DISPONIBLES")
    print("\nPróximos pasos:")
    print("   1. Implementar interpolación CHIRPS → ERA5")
    print("   2. Calcular métricas de validación cruzada")
    print("   3. Generar visualizaciones comparativas")
    print("   4. Actualizar documentación con hallazgos")